In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Fetch Data 

In [2]:
df = pd.read_csv("./SalesFINAL12312016.csv")
df.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/1/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,1/2/2016,750,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,1/3/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,1/8/2016,750,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,1/9/2016,375,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [3]:
#Determine the size of the dataframe
df.shape

(1048575, 14)

This implies we have 1,048,575 (near a million) rows and 14 columns(features) in our dataframe.

To perform **Demand Forecasting**, we will take a random sample from the dataframe, of the size = 40,000

In [4]:
df = df.sample(n = 40000)

## DATA ANALYSIS

In [5]:
print("\nSales Columns:")
print(df.columns.tolist())


Sales Columns:
['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification', 'ExciseTax', 'VendorNo', 'VendorName']


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 886309 to 233293
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   InventoryId     40000 non-null  object 
 1   Store           40000 non-null  int64  
 2   Brand           40000 non-null  int64  
 3   Description     40000 non-null  object 
 4   Size            40000 non-null  object 
 5   SalesQuantity   40000 non-null  int64  
 6   SalesDollars    40000 non-null  float64
 7   SalesPrice      40000 non-null  float64
 8   SalesDate       40000 non-null  object 
 9   Volume          40000 non-null  int64  
 10  Classification  40000 non-null  int64  
 11  ExciseTax       40000 non-null  float64
 12  VendorNo        40000 non-null  int64  
 13  VendorName      40000 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 4.6+ MB


In [7]:
df.dtypes

InventoryId        object
Store               int64
Brand               int64
Description        object
Size               object
SalesQuantity       int64
SalesDollars      float64
SalesPrice        float64
SalesDate          object
Volume              int64
Classification      int64
ExciseTax         float64
VendorNo            int64
VendorName         object
dtype: object

**Observation**: We observe that, the SalesDate feature is of type "object". We need SalesDate later, to predict the SalesQuantity, so we need to handle it's data type.

In [8]:
df.describe()

,Store,Brand,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,39.761350,12130.663000,2.320700,31.074697,15.422475,954.832400,1.412625,1.318562,7036.539850
std,24.314159,12401.273783,3.509503,59.756713,13.395127,728.465541,0.492313,3.351901,8674.887707
min,1.000000,58.000000,1.000000,0.990000,0.490000,50.000000,1.000000,0.010000,105.000000
25%,15.000000,3663.000000,1.000000,10.990000,8.990000,750.000000,1.000000,0.160000,3252.000000
50%,38.000000,6269.000000,1.000000,17.990000,12.990000,750.000000,1.000000,0.680000,4425.000000
75%,64.000000,17875.250000,2.000000,31.980000,18.990000,1500.000000,2.000000,1.570000,9552.000000
max,79.000000,90025.000000,157.000000,3058.980000,999.990000,18000.000000,2.000000,187.420000,173357.000000


**Observation**: We observe that there is a huge deviation between our minimum and maximum SalesQuantity. This will also affect our predictions. We will handle these extreme values(*Outliers*), later in this notebook.

In [9]:
df.isnull().sum()

InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64

**Observation**: We don't have any null values in our DataFrame. So far good to GO!

In [10]:
df.Description.value_counts()

Smirnoff 80 Proof               268
Kahlua                          262
Jim Beam                        259
Capt Morgan Spiced Rum          256
Jagermeister Liqueur            254
                               ... 
Folie A Deux Cab Svgn             1
Ch Pape Clement Pess leognan      1
Heitz Znfdl Ink Grade Vyd         1
Fifty Shades Of Grey Wh Silk      1
Russo Limoncello                  1
Name: Description, Length: 3907, dtype: int64

**Observation**: In our dataframe, for some brands, we've enough number of rows to predict their sales, but for some brands we only have one row, which may impact our predictions.

## DATA PREPROCESSING

In [11]:
df['VendorName'] = df['VendorName'].str.strip()
df['Description'] = df['Description'].str.strip()

Convert the SalesDate to Date time Object

In [12]:
df['SalesDate'] = pd.to_datetime(df['SalesDate'])

In [13]:
df.dtypes

InventoryId               object
Store                      int64
Brand                      int64
Description               object
Size                      object
SalesQuantity              int64
SalesDollars             float64
SalesPrice               float64
SalesDate         datetime64[ns]
Volume                     int64
Classification             int64
ExciseTax                float64
VendorNo                   int64
VendorName                object
dtype: object

Extract the year, month and day from the SalesDate

In [14]:
df['year'] = df['SalesDate'].dt.year
df['month'] = df['SalesDate'].dt.month
df['day'] = df['SalesDate'].dt.day

In [15]:
df

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
886309,77_TAMWORTH_24096,77,24096,Gallo Copperidge Chard Cal,1.5L,6,41.94,6.99,2016-01-07,1500,2,1.35,3252,E & J GALLO WINERY,2016,1,7
952170,1_HARDERSFIELD_6649,1,6649,Wente Morning Fog Chard,750mL,1,11.99,11.99,2016-02-23,750,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,2016,2,23
260656,33_HORNSEY_3609,33,3609,Smirnoff Vanilla Vodka,50mL,1,0.99,0.99,2016-01-15,50,1,0.05,3960,DIAGEO NORTH AMERICA INC,2016,1,15
946833,1_HARDERSFIELD_3837,1,3837,Skyy Vodka,50mL,4,3.96,0.99,2016-02-13,50,1,0.21,11567,CAMPARI AMERICA,2016,2,13
870870,76_DONCASTER_3702,76,3702,Kinky Vodka,50mL,3,2.97,0.99,2016-01-22,50,1,0.16,7153,PINE STATE TRADING CO,2016,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444892,49_GARIGILL_4313,49,4313,Sauza Blue Silver Tequila,1.75L,1,29.99,29.99,2016-01-30,1750,1,1.84,12546,JIM BEAM BRANDS COMPANY,2016,1,30
19449,10_HORNSEY_16326,10,16326,Black Box Cab Svgn,500mL,2,9.98,4.99,2016-01-08,500,2,0.15,1392,CONSTELLATION BRANDS INC,2016,1,8
872077,76_DONCASTER_3847,76,3847,Grey Goose Vodka,50mL,2,7.98,3.99,2016-01-08,50,1,0.10,480,BACARDI USA INC,2016,1,8
33809,10_HORNSEY_8172,10,8172,DK Burst Blustery Peppermint,50mL,4,3.96,0.99,2016-01-12,50,1,0.21,12546,JIM BEAM BRANDS COMPANY,2016,1,12


Drop the following Columns:
1) InventoryId: because each row has different ID 
2) SalesDate: because we've already extracted the required info from this feature in year, month, and day columns.
3) Size: because it has some ambiguity in it. 
4) SalesDollars: represents redundant data in terms of SalesPrice * SalesQuantity

In [16]:
df.drop(['InventoryId','SalesDate', 'Size'], inplace=True, axis=1)

In [17]:
df.head()

,Store,Brand,Description,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
886309,77,24096,Gallo Copperidge Chard Cal,6,41.94,6.99,1500,2,1.35,3252,E & J GALLO WINERY,2016,1,7
952170,1,6649,Wente Morning Fog Chard,1,11.99,11.99,750,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,2016,2,23
260656,33,3609,Smirnoff Vanilla Vodka,1,0.99,0.99,50,1,0.05,3960,DIAGEO NORTH AMERICA INC,2016,1,15
946833,1,3837,Skyy Vodka,4,3.96,0.99,50,1,0.21,11567,CAMPARI AMERICA,2016,2,13
870870,76,3702,Kinky Vodka,3,2.97,0.99,50,1,0.16,7153,PINE STATE TRADING CO,2016,1,22


### Removing the Outliers

In [18]:
z_scores = (df[['SalesQuantity', 'SalesDollars', 'SalesPrice', 'Volume', 'ExciseTax']] - df[['SalesQuantity', 'SalesDollars', 'SalesPrice', 'Volume', 'ExciseTax']].mean()) / df[['SalesQuantity', 'SalesDollars', 'SalesPrice', 'Volume', 'ExciseTax']].std()
threshold = 3
outliers = df[(np.abs(z_scores) > threshold).any(axis=1)]
df_cleaned = df[(np.abs(z_scores) <= threshold).all(axis=1)]

In [19]:
df_cleaned

,Store,Brand,Description,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo,VendorName,year,month,day
886309,77,24096,Gallo Copperidge Chard Cal,6,41.94,6.99,1500,2,1.35,3252,E & J GALLO WINERY,2016,1,7
952170,1,6649,Wente Morning Fog Chard,1,11.99,11.99,750,2,0.11,2000,SOUTHERN WINE & SPIRITS NE,2016,2,23
260656,33,3609,Smirnoff Vanilla Vodka,1,0.99,0.99,50,1,0.05,3960,DIAGEO NORTH AMERICA INC,2016,1,15
946833,1,3837,Skyy Vodka,4,3.96,0.99,50,1,0.21,11567,CAMPARI AMERICA,2016,2,13
870870,76,3702,Kinky Vodka,3,2.97,0.99,50,1,0.16,7153,PINE STATE TRADING CO,2016,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444892,49,4313,Sauza Blue Silver Tequila,1,29.99,29.99,1750,1,1.84,12546,JIM BEAM BRANDS COMPANY,2016,1,30
19449,10,16326,Black Box Cab Svgn,2,9.98,4.99,500,2,0.15,1392,CONSTELLATION BRANDS INC,2016,1,8
872077,76,3847,Grey Goose Vodka,2,7.98,3.99,50,1,0.10,480,BACARDI USA INC,2016,1,8
33809,10,8172,DK Burst Blustery Peppermint,4,3.96,0.99,50,1,0.21,12546,JIM BEAM BRANDS COMPANY,2016,1,12


In [20]:
df_cleaned.shape

(37835, 14)

**Observation** : Earlier we had 40,000 rows, after removing the outliers, the number of rows has reduced.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 886309 to 233293
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Store           40000 non-null  int64  
 1   Brand           40000 non-null  int64  
 2   Description     40000 non-null  object 
 3   SalesQuantity   40000 non-null  int64  
 4   SalesDollars    40000 non-null  float64
 5   SalesPrice      40000 non-null  float64
 6   Volume          40000 non-null  int64  
 7   Classification  40000 non-null  int64  
 8   ExciseTax       40000 non-null  float64
 9   VendorNo        40000 non-null  int64  
 10  VendorName      40000 non-null  object 
 11  year            40000 non-null  int64  
 12  month           40000 non-null  int64  
 13  day             40000 non-null  int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 4.6+ MB


### Applying One-Hot Encoding
To enable the model to effectively process textual data (*product descriptions*), one-hot encoding is applied. This process converts description strings into individual boolean columns, where each column represents a specific type of product. 

In [22]:
description_dummies = pd.get_dummies(df_cleaned['Description'])
vendorname_dummies = pd.get_dummies(df_cleaned['VendorName'])
dfx = pd.concat([df_cleaned, description_dummies, vendorname_dummies], axis=1)
dfx.drop(['Description', 'VendorName'], axis=1, inplace=True)
dfx.head()

,Store,Brand,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
886309,77,24096,6,41.94,6.99,1500,2,1.35,3252,2016,...,0,0,0,0,0,0,0,0,0,0
952170,1,6649,1,11.99,11.99,750,2,0.11,2000,2016,...,0,0,0,0,0,0,0,0,0,0
260656,33,3609,1,0.99,0.99,50,1,0.05,3960,2016,...,0,0,0,0,0,0,0,0,0,0
946833,1,3837,4,3.96,0.99,50,1,0.21,11567,2016,...,0,0,0,0,0,0,0,0,0,0
870870,76,3702,3,2.97,0.99,50,1,0.16,7153,2016,...,0,0,0,0,0,0,0,0,0,0


The number of columns in our data has increased, reflecting the one-hot encoding

In [23]:
dfx

,Store,Brand,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
886309,77,24096,6,41.94,6.99,1500,2,1.35,3252,2016,...,0,0,0,0,0,0,0,0,0,0
952170,1,6649,1,11.99,11.99,750,2,0.11,2000,2016,...,0,0,0,0,0,0,0,0,0,0
260656,33,3609,1,0.99,0.99,50,1,0.05,3960,2016,...,0,0,0,0,0,0,0,0,0,0
946833,1,3837,4,3.96,0.99,50,1,0.21,11567,2016,...,0,0,0,0,0,0,0,0,0,0
870870,76,3702,3,2.97,0.99,50,1,0.16,7153,2016,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444892,49,4313,1,29.99,29.99,1750,1,1.84,12546,2016,...,0,0,0,0,0,0,0,0,0,0
19449,10,16326,2,9.98,4.99,500,2,0.15,1392,2016,...,0,0,0,0,0,0,0,0,0,0
872077,76,3847,2,7.98,3.99,50,1,0.10,480,2016,...,0,0,0,0,0,0,0,0,0,0
33809,10,8172,4,3.96,0.99,50,1,0.21,12546,2016,...,0,0,0,0,0,0,0,0,0,0



A value of true in the column indicates that the respective row refers to that product.

### Applying Min-Max Scaling
Min-Max Scaling is done to linearly scale the values of *Sales Dollars* and *Sales Price* between 0 and 1.

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dfx.loc[:, "SalesDollars"] = scaler.fit_transform(df_cleaned[["SalesDollars"]].values.reshape(-1, 1))
dfx.loc[:, "SalesPrice"] = scaler.fit_transform(df_cleaned[["SalesPrice"]].values.reshape(-1, 1))

In [25]:
dfx

,Store,Brand,SalesQuantity,SalesDollars,SalesPrice,Volume,Classification,ExciseTax,VendorNo,year,...,VINEXTRA INC,VINEYARD BRANDS INC,VINILANDIA USA,VRANKEN AMERICA,WALPOLE MTN VIEW WINERY,WEIN BAUER INC,WESTERN SPIRITS BEVERAGE CO,WILLIAM GRANT & SONS INC,WINE GROUP INC,ZORVINO VINEYARDS
886309,77,24096,6,0.195971,0.111111,1500,2,1.35,3252,2016,...,0,0,0,0,0,0,0,0,0,0
952170,1,6649,1,0.052642,0.203704,750,2,0.11,2000,2016,...,0,0,0,0,0,0,0,0,0,0
260656,33,3609,1,0.000000,0.000000,50,1,0.05,3960,2016,...,0,0,0,0,0,0,0,0,0,0
946833,1,3837,4,0.014213,0.000000,50,1,0.21,11567,2016,...,0,0,0,0,0,0,0,0,0,0
870870,76,3702,3,0.009475,0.000000,50,1,0.16,7153,2016,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444892,49,4313,1,0.138783,0.537037,1750,1,1.84,12546,2016,...,0,0,0,0,0,0,0,0,0,0
19449,10,16326,2,0.043023,0.074074,500,2,0.15,1392,2016,...,0,0,0,0,0,0,0,0,0,0
872077,76,3847,2,0.033451,0.055556,50,1,0.10,480,2016,...,0,0,0,0,0,0,0,0,0,0
33809,10,8172,4,0.014213,0.000000,50,1,0.21,12546,2016,...,0,0,0,0,0,0,0,0,0,0


## PREDICTING SALES DEMAND

In [26]:
y=dfx['SalesQuantity']
x=dfx.drop(['SalesQuantity'],axis=1)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [28]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()

Training the model

In [29]:
model.fit(x_train,y_train)

RandomForestClassifier()

Evaluating the model

In [30]:
model.score(x_test,y_test)

0.920181837403531

In [31]:
import warnings
warnings.filterwarnings('ignore')

This function returns the index of the column associated with the provided description after one hot encoding

In [32]:
def get_loc_index(description):
    try:
        return x.columns.get_loc(description)
    except KeyError:
        return -1 

In [33]:
def predict_sales_quantity(VendorName, Description, Store, Brand, Volume, SalesPrice, Classification, ExciseTax, VendorNo,year,month,day):
    X = np.zeros(len(x.columns))
    X[0] = Store
    X[1] = Brand
    X[2] = Volume
    X[3] = SalesPrice
    X[4] = Classification
    X[5] = ExciseTax
    X[6] = VendorNo
    X[7] = year
    X[8] = month
    X[9]= day

    # Set the index corresponding to Description column to 1 if loc_index >= 0
    loc_index = get_loc_index(Description)
    if loc_index >= 0:
        X[loc_index] = 1

    # Make prediction
    predicted_sales_quantity = model.predict([X])[0]

    return predicted_sales_quantity

In [34]:
predict_sales_quantity('ULTRA BEVERAGE COMPANY LLP','Cecchi Sangiovese',1,18013,750,6.99,2,0.56,9165,2017,1,21)

4